<a href="https://colab.research.google.com/github/ShaunakSen/Contextual-hyperlinks/blob/master/Dataset_extraction_%2B_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/google-research-datasets/wiki-links/raw/master/data-00001.gz

--2019-08-18 13:16:32--  https://github.com/google-research-datasets/wiki-links/raw/master/data-00001.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research-datasets/wiki-links/master/data-00001.gz [following]
--2019-08-18 13:16:33--  https://raw.githubusercontent.com/google-research-datasets/wiki-links/master/data-00001.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17111634 (16M) [application/octet-stream]
Saving to: ‘data-00001.gz’

data-00001.gz       100%[===================>]  16.32M   101MB/s    in 0.2s    

2019-08-18 13:16:34 (101 MB/s) - ‘data-00001.gz’ saved [17111634/17111

In [0]:
import urllib.request
import os
import zipfile
import re

In [0]:
!gzip -d /content/data-00001.gz

In [1]:
from google.colab import files

uploaded = files.upload()

Saving data-00001 to data-00001


In [0]:
f = open('./data-00001', 'r')

lines = f.readlines()

In [20]:
def process_url(url):
  """
  function to determine if to keep a URL or not
  """
  # check start of url - http or https
  if not url.startswith("http"):
    return False
  # check first characters and extensions
  if re.match(pattern='(http|https)://\w.*', string=url) is None or url.endswith("pdf") or url.endswith("doc") or url.endswith("docx"):
    return False
  # all passed return True
  return True

total_urls = 0 # etsimate of total source urls in the dataset: may contain duplicates 

for line in lines:
  if (line.startswith("URL")):
    url = line.split()[1]
    # check if URL valid
    if process_url(url):
      total_urls+=1


print ("Total URLs:", total_urls)

Total URLs: 99494


In [30]:
mention_text = "MENTION	Los Angeles Philharmonic	17030	http://en.wikipedia.org/wiki/Los_Angeles_Philharmonic"

mention_text = mention_text.split()

print (mention_text)

mention_token = " ".join(mention_text[1:len(mention_text)-2])

print (mention_token)

target = mention_text[len(mention_text)-1]
print (target)

['MENTION', 'Los', 'Angeles', 'Philharmonic', '17030', 'http://en.wikipedia.org/wiki/Los_Angeles_Philharmonic']
Los Angeles Philharmonic
http://en.wikipedia.org/wiki/Los_Angeles_Philharmonic


In [0]:
# [[mention_text, source, target], ...]

dataset = []

for i in range(len(lines)):
  if (lines[i].startswith("URL")):
    url = lines[i].split()[1]
    # check if URL valid
    if process_url(url):
      # look at the MENTION tags
      curr_idx = i+1
      next_line = lines[curr_idx]
      while next_line.startswith("MENTION"):
        # MENTION detected
        # split by ' '
        mention_text = next_line.split()
        # get mention tokens
        mention_token = " ".join(mention_text[1:len(mention_text)-2])
        # get target link
        target = mention_text[len(mention_text)-1]
        # append in form: [mention_text, source, target]
        dataset.append([mention_token, url, target])
        curr_idx += 1
        next_line = lines[curr_idx]


In [36]:
dataset[:10]

[['Los Angeles Philharmonic',
  'http://0-m-f-g.blogspot.com/2010/05/walt-disney-concert-hall.html',
  'http://en.wikipedia.org/wiki/Los_Angeles_Philharmonic'],
 ['Los Angeles Master Chorale',
  'http://0-m-f-g.blogspot.com/2010/05/walt-disney-concert-hall.html',
  'http://en.wikipedia.org/wiki/Los_Angeles_Master_Chorale'],
 ['Walt Disney',
  'http://0-m-f-g.blogspot.com/2010/05/walt-disney-concert-hall.html',
  'http://en.wikipedia.org/wiki/Walt_Disney'],
 ['Evo',
  'http://0-one.org/2012/07/14/ouya-99-android-game-console/',
  'http://en.wikipedia.org/wiki/EVO_Smart_Console'],
 ['Groupthink',
  'http://0.yogins.com/2012/01/30/groupthink-and-centralized-bathrooms-with-knowledge-spillovers/',
  'http://en.wikipedia.org/wiki/Groupthink'],
 ['Knowledge Spillovers',
  'http://0.yogins.com/2012/01/30/groupthink-and-centralized-bathrooms-with-knowledge-spillovers/',
  'http://en.wikipedia.org/wiki/Knowledge_spillover'],
 ['Latin',
  'http://00000000.ca/',
  'http://en.wikipedia.org/wiki/Lat

In [34]:
print (len(dataset))

373961


In [38]:
print ("MENTIONs per link:", len(dataset)/total_urls)

MENTIONs per link: 3.7586286610247854
